In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
from IPython.core.debugger import Pdb

# Definition of parameters

In [31]:
import timeit

a_cuda = torch.rand(1000, device='cuda')
b_cuda = torch.rand((1000, 1000), device='cuda')
a_cpu = torch.rand(1000, device='cpu')
b_cpu = torch.rand((1000, 1000), device='cpu')
 
#print('cuda', timeit.timeit(lambda: a_cuda @ b_cuda, number=100_000))
# print('cpu', timeit.timeit(lambda: a_cpu @ b_cpu, number=100_000))

#print('cuda', timeit.timeit(lambda: b_cuda @ b_cuda, number=100_000))
# print('cpu', timeit.timeit(lambda: b_cpu @ b_cpu, number=10_000))



In [32]:

# # b_cpu = torch.rand((1000, 1000), device='cpu')
# b_cuda = torch.rand((1000, 1000), device='cuda')

# # print('cpu', timeit.timeit(lambda: b_cpu @ b_cpu, number=100))
# print('cuda', timeit.timeit(lambda: b_cuda @ b_cuda, number=10_000))

In [33]:
# def test_cpu():
#     a_cpu = torch.rand(1000, device='cpu')
#     b_cpu = torch.rand((1000, 1000), device='cpu')
#     a_cpu @ b_cpu
# def test_cuda():
#     a_cuda = torch.rand(1000, device='cuda')
#     b_cuda = torch.rand((1000, 1000), device='cuda')
#     a_cuda @ b_cuda

# print('cpu', timeit.timeit(lambda: test_cpu(), number=1000))
# print('cuda', timeit.timeit(lambda: test_cuda(), number=1000))

In [34]:
import os
import shutil

device = 'cuda'
print(device)
learning_rate = 0.00001
batch_size = 2
experiment_name = 'test_experiment'
# the directory experiment_name exists, delete it
# if os.path.exists(experiment_name):
#     shutil.rmtree(experiment_name)


cuda


# Data loading

In [35]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, csv):
        # read the csv file
        self.df = pd.read_csv(csv, sep='\s+')
        # self.df = self.df.dropna(axis=0)
        # save cols
        self.input_cols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
        self.output_cols = ['MEDV']
        


    def __len__(self):
        # here i will return the number of samples in the dataset
        return len(self.df)


    def __getitem__(self, idx):
        # here i will load the file in position idx
        cur_sample = self.df.iloc[idx]
        if cur_sample.isna().any():
            Pdb()()
        # split in input / ground-truth
        cur_sample_x = cur_sample[self.input_cols]
        cur_sample_y = cur_sample[self.output_cols]
        # convert to torch format
        cur_sample_x = torch.tensor(cur_sample_x.tolist())
        cur_sample_y = torch.tensor(cur_sample_y.tolist())
        # return values
        return cur_sample_x, cur_sample_y


<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_506135/2631637122.py:5: SyntaxWarning: invalid escape sequence '\s'
  self.df = pd.read_csv(csv, sep='\s+')


In [36]:
# try to use the dataset
ds = Dataset('../datasets/BostonHousingDataset/train.csv')
# get first item
xx,yy = ds.__getitem__(0)
# print shapes
print(xx.shape)
print(yy.shape)

torch.Size([13])
torch.Size([1])


In [37]:
# create train and validation datasets
train_ds = Dataset('../datasets/BostonHousingDataset/train.csv')
val_ds =  Dataset('../datasets/BostonHousingDataset/val.csv')

In [38]:
# create train dataloader
train_dl = torch.utils.data.DataLoader(
    train_ds,
    batch_size = batch_size,
    drop_last = True,
    shuffle = True,
    num_workers = 4
)
# create validation dataloader
val_dl = torch.utils.data.DataLoader(
    val_ds,
    batch_size = batch_size,
    drop_last = False,
    shuffle = False,
    num_workers = 4
)

train_ds.__getitem__(1)

(tensor([-0.4176,  1.8728, -1.0734, -0.2726, -0.6107,  0.5861, -0.4329,  0.9208,
         -0.5230, -0.2271, -0.3956,  0.4411, -0.7672]),
 tensor([0.1488]))

# Network definition

In [39]:
# define network

class Net(nn.Module):

    def __init__(self):
        # initialize super class
        super(Net, self).__init__()
        self.layer1 = nn.Linear(13,128)
        self.layer2 = nn.ReLU()
        self.layer3 = nn.Linear(128,64)
        self.layer4 = nn.ReLU()
        self.layer5 = nn.Linear(64,32)
        self.layer6 = nn.ReLU()
        self.layer7 = nn.Linear(32,16)
        self.layer8 = nn.ReLU()
        self.layer9 = nn.Linear(16, 1)


    def forward(self, x):
        # apply layers in cascade
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)
        # return output
        return x


In [40]:
# let's test the network
net = Net()

# define random batch of 10 elements
inp = torch.rand(10, 13)

# forward
out = net(inp)

# let's print the shape
print(' Input shape is', inp.shape)
print('Output shape is', out.shape)

 Input shape is torch.Size([10, 13])
Output shape is torch.Size([10, 1])


In [41]:
# let's move the network in GPU
net.to(device)

# define random batch of 10 elements
inp = torch.rand(10, 13)

# move the batch in GPU
inp = inp.to(device)

# get the output
out = net(inp)

# let's print the shape
print(' Input shape is', inp.shape)
print('Output shape is', out.shape)

 Input shape is torch.Size([10, 13])
Output shape is torch.Size([10, 1])


# Define validation routine

In [42]:
# create validation routine
def validate(net, dl):
    # get final score
    score = 0
    # set network in eval mode
    net.eval()
    # at the end of epoch, validate model
    for inp, gt in dl:
        # move batch to gpu
        inp = inp.to(device)
        gt = gt.to(device)
        # get output
        with torch.no_grad():
            out = net(inp)
        # compare with gt
        cur_score = F.l1_loss(out, gt)
        # append
        score += cur_score 
    # at the end, average over batches
    score /= len(dl)
    # set network in training mode
    net.train()
    # return score
    return score
        
        

# Train

In [43]:
from tqdm import tqdm
# define optimizer
optimizer = torch.optim.Adam(params=net.parameters(), lr=learning_rate)

# define summary writer
writer = SummaryWriter(experiment_name)

# initialize iteration number
n_iter = 0

# define best validation value
best_val = None

# for each epoch
for cur_epoch in tqdm(range(250)):
    # plot current epoch
    writer.add_scalar("epoch", cur_epoch, n_iter)
    # for each batch
    for inp, gt in train_dl:
        # move batch to gpu
        inp = inp.to(device)
        gt = gt.to(device)
        # reset gradients
        optimizer.zero_grad()
        # get output
        out = net(inp)
        # compute loss
        loss = F.l1_loss(out, gt)
        # compute backward
        loss.backward()
        # update weights
        optimizer.step()
        # plot
        writer.add_scalar("loss", loss.item(), n_iter)
        n_iter += 1
        
    # at the end, validate model
    cur_val = validate(net, val_dl)
    # plot validation
    writer.add_scalar("val", loss.item(), n_iter)
    # check if it is the best model so far
    if best_val is None or cur_val > best_val:
        # define new best val
        best_val = cur_val
        # save current model as best
        torch.save({
            'net': net.state_dict(),
            'opt': optimizer.state_dict(),
            'epoch': cur_epoch
        }, experiment_name + '_best.pth')
        # save last model
        torch.save({
            'net': net.state_dict(),
            'opt': optimizer.state_dict(),
            'epoch': cur_epoch
        }, experiment_name + '_last.pth')
    

  0%|          | 1/250 [00:00<02:15,  1.84it/s]

 32%|███▏      | 79/250 [00:39<01:25,  2.00it/s]Exception ignored in: <function _releaseLock at 0x7208a0db7ce0>
Traceback (most recent call last):
  File "/usr/lib/python3.12/logging/__init__.py", line 243, in _releaseLock
    def _releaseLock():
    
KeyboardInterrupt: 
 41%|████      | 103/250 [00:51<01:14,  1.96it/s]

# Test

In [ ]:
# create test dataset
test_ds =  Dataset('data/test.csv')

# create dataloader
test_dl = torch.utils.data.DataLoader(
    test_ds,
    batch_size = batch_size,
    drop_last = False,
    shuffle = False,
    num_workers = 4
)

/tmp/ipykernel_506135/2631637122.py:5: SyntaxWarning: invalid escape sequence '\s'
  self.df = pd.read_csv(csv, sep='\s+')


FileNotFoundError: [Errno 2] No such file or directory: 'data/test.csv'

In [ ]:
# load best network
state = torch.load(experiment_name + '_best.pth')
net.load_state_dict(state['net'])

<All keys matched successfully>

In [ ]:
test_value = validate(net, test_dl).item()

In [ ]:
print(f'The model scored a MAE of {test_value:0.04f} over the testset.')

The model scored a MAE of 0.6519 over the testset.
